In [1]:
# imports
import pandas as pd
import numpy as np
from env import host, user, password
import acquire
import prepare

In [2]:
# use my functions to acquire telco_churn 
df = acquire.get_telco_data()

In [3]:
# let's see the value counts...
col_list = df.columns

In [4]:
col_list

Index(['payment_type_id', 'internet_service_type_id', 'contract_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type'],
      dtype='object')

In [5]:
# let's look at all columns value_counts
for col in col_list:
    print(col)
    print(df[col].value_counts())
    print('--------------------------------')

payment_type_id
1    2365
2    1612
3    1544
4    1522
Name: payment_type_id, dtype: int64
--------------------------------
internet_service_type_id
2    3096
1    2421
3    1526
Name: internet_service_type_id, dtype: int64
--------------------------------
contract_type_id
1    3875
3    1695
2    1473
Name: contract_type_id, dtype: int64
--------------------------------
customer_id
6435-SRWBJ    1
5707-ZMDJP    1
4947-DSMXK    1
2342-CKIAO    1
9351-LZYGF    1
             ..
0769-MURVM    1
6166-ILMNY    1
7602-DBTOU    1
4176-RELJR    1
5803-NQJZO    1
Name: customer_id, Length: 7043, dtype: int64
--------------------------------
gender
Male      3555
Female    3488
Name: gender, dtype: int64
--------------------------------
senior_citizen
0    5901
1    1142
Name: senior_citizen, dtype: int64
--------------------------------
partner
No     3641
Yes    3402
Name: partner, dtype: int64
--------------------------------
dependents
No     4933
Yes    2110
Name: dependents, dtype: int64

In [6]:
# let's see what we're working with...
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [7]:
df['customer_id'].nunique()

7043

In [8]:
# reset the customer_id to be the index
df = df.set_index('customer_id')

In [9]:
df.index

Index(['0016-QLJIS', '0017-DINOC', '0019-GFNTW', '0056-EPFBG', '0078-XZMHT',
       '0114-IGABW', '0123-CRBRT', '0191-EQUUH', '0214-JHPFW', '0229-LFJAF',
       ...
       '9837-FWLCH', '9860-LISIZ', '9867-NNXLC', '9929-PLVPA', '9932-WBWIK',
       '9962-BFPDU', '9967-ATRFS', '9970-QBCDA', '9975-SKRNR', '9986-BONCE'],
      dtype='object', name='customer_id', length=7043)

In [10]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,,
0016-QLJIS,2,1,3,Female,0,Yes,Yes,65,Yes,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
0017-DINOC,4,1,3,Male,0,No,No,54,No,No phone service,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
0019-GFNTW,3,1,3,Female,0,No,No,56,No,No phone service,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
0056-EPFBG,4,1,3,Male,0,Yes,Yes,20,No,No phone service,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
0078-XZMHT,3,1,3,Male,0,Yes,No,72,Yes,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [11]:
df.index.nunique()

7043

In [12]:
# are there any null values / whitespace?
# This replaces empty cells with nan (null values)
df = df.replace(' ', np.nan)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0016-QLJIS to 9986-BONCE
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   gender                    7043 non-null   object 
 4   senior_citizen            7043 non-null   int64  
 5   partner                   7043 non-null   object 
 6   dependents                7043 non-null   object 
 7   tenure                    7043 non-null   int64  
 8   phone_service             7043 non-null   object 
 9   multiple_lines            7043 non-null   object 
 10  online_security           7043 non-null   object 
 11  online_backup             7043 non-null   object 
 12  device_protection         7043 non-null   object 
 13  tech_support              7043 non-null   object 
 14

In [14]:
# drop the nulls...
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0016-QLJIS to 9986-BONCE
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7032 non-null   int64  
 1   internet_service_type_id  7032 non-null   int64  
 2   contract_type_id          7032 non-null   int64  
 3   gender                    7032 non-null   object 
 4   senior_citizen            7032 non-null   int64  
 5   partner                   7032 non-null   object 
 6   dependents                7032 non-null   object 
 7   tenure                    7032 non-null   int64  
 8   phone_service             7032 non-null   object 
 9   multiple_lines            7032 non-null   object 
 10  online_security           7032 non-null   object 
 11  online_backup             7032 non-null   object 
 12  device_protection         7032 non-null   object 
 13  tech_support              7032 non-null   object 
 14

In [15]:
# convert payment types to 1:auto-pay, 0:not auto-pay
df['auto_pay'] = df['payment_type_id'].replace({1:0, 2:0, 3:1, 4:1})

In [16]:
# create DSL column where 1:has DSL, 0:No DSL
df['dsl'] = df['internet_service_type_id'].replace({1:1, 2:0, 3:0})

In [17]:
# create Fiber column where 1:has Fiber service, 0:No Fiber
df['fiber'] = df['internet_service_type_id'].replace({1:0, 2:1, 3:0})

In [18]:
# create Has Internet column where 1:Has Internet 0:No internet service
df['has_internet'] = df['internet_service_type_id'].replace({1:1, 2:1, 3:0})

In [19]:
# separte contract_type_id into three columns...
# create m2m column where 1:Month-to-Month service, 2:contract
df['m2m'] = df['contract_type_id'].replace({1:1, 2:0, 3:0})

In [20]:
# create one_year column where 1:One year contract, 0:no contract, or m2m
df['one_year'] = df['contract_type_id'].replace({1:0, 2:1, 3:0})

In [21]:
# create teo_year column where 1:two year contract, 0:less than 2 year contract
df['two_year'] = df['contract_type_id'].replace({1:0, 2:0, 3:1})

In [22]:
# create has contract column where 1:has contract, 0:no contract
df['has_contract'] = df['contract_type_id'].replace({1:0, 2:1, 3:1})

In [23]:
# create column to convert gender to int 1:male, 0:female
df['is_male'] = df['gender'].replace({'Male':1, 'Female':0})

In [24]:
# create reusable dictionary for replacing 'No', 'Yes', 'No internet service', 'No phone service'
rep_dict = {
    'No': 0,
    'Yes': 1,
    'No internet service': 0,
    'No phone service': 0
}

In [25]:
# create has_partner column where 1:has partner, 0:no partner
df['has_partner'] = df['partner'].replace(rep_dict)

In [26]:
# create has_dep column where 1:has dependents, 0:no dependents
df['has_dep'] = df['dependents'].replace(rep_dict)

In [27]:
# better identify tenure in months by renaming column...
df['tenure_months'] = df['tenure']

In [28]:
# create has_phone column where 1:has phone, 0:no phone
df['has_phone'] = df['phone_service'].replace(rep_dict)

In [29]:
# create multi_phone column where 1:multiple phone lines, 0:One or fewer phone lines
df['multi_phone'] = df['multiple_lines'].replace(rep_dict)

In [30]:
# create security column where 1:has online security, 2:no security
df['has_security'] = df['online_security'].replace(rep_dict)

In [31]:
# create has_backup column where 1:has online backup, 0:no backup
df['has_backup'] = df['online_backup'].replace(rep_dict)

In [32]:
# create has_protection column where 1:has device protection, 0:no device protection
df['has_protection'] = df['device_protection'].replace(rep_dict)

In [33]:
# create has_support column where 1:has tech support, 0:no tech support
df['has_support'] = df['tech_support'].replace(rep_dict)

In [34]:
# create stream_tv column where 1:streams tv, 0:no streaming tv
df['stream_tv'] = df['streaming_tv'].replace(rep_dict)

In [35]:
# create stream_movies column where 1:streams movies, 0:no streaming movies
df['stream_movies'] = df['streaming_movies'].replace(rep_dict)

In [36]:
# create has_paperless column where 1:has paperless billing, 0:no paperless billing
df['has_paperless'] = df['paperless_billing'].replace(rep_dict)

In [37]:
# convert total_charges to float
df['total_charges'] = df['total_charges'].astype(float)

In [38]:
# create has_churn column where 1:has churn, 0:no churn
df['has_churn'] = df['churn'].replace(rep_dict)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0016-QLJIS to 9986-BONCE
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7032 non-null   int64  
 1   internet_service_type_id  7032 non-null   int64  
 2   contract_type_id          7032 non-null   int64  
 3   gender                    7032 non-null   object 
 4   senior_citizen            7032 non-null   int64  
 5   partner                   7032 non-null   object 
 6   dependents                7032 non-null   object 
 7   tenure                    7032 non-null   int64  
 8   phone_service             7032 non-null   object 
 9   multiple_lines            7032 non-null   object 
 10  online_security           7032 non-null   object 
 11  online_backup             7032 non-null   object 
 12  device_protection         7032 non-null   object 
 13  tech_support              7032 non-null   object 
 14

In [40]:
# now I have my wide DataFrame, let's prep round 2

In [41]:
# I want to make sure I keep all columns in this df, but I want to eliminate
# some for me to work with. That way, if I need a visual check, I can look back, but
# I want have as much out of my way as possible for me to work with

In [42]:
# let's get a list of columns to drop
list(df.select_dtypes('object').columns)

['gender',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [43]:
# all object columns in a variable to drop
obj_cols = list(df.select_dtypes('object').columns)

In [44]:
# other columns to drop...
other_drops = ['payment_type_id', 'internet_service_type_id', 'contract_type_id']

In [49]:
# combine all drops into 1 list
all_drops = obj_cols + other_drops

In [50]:
# return a new, clean dataframe
df_1 = df.drop(columns=all_drops)

In [51]:
# what's it look like?
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0016-QLJIS to 9986-BONCE
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   senior_citizen   7032 non-null   int64  
 1   tenure           7032 non-null   int64  
 2   monthly_charges  7032 non-null   float64
 3   total_charges    7032 non-null   float64
 4   auto_pay         7032 non-null   int64  
 5   dsl              7032 non-null   int64  
 6   fiber            7032 non-null   int64  
 7   has_internet     7032 non-null   int64  
 8   m2m              7032 non-null   int64  
 9   one_year         7032 non-null   int64  
 10  two_year         7032 non-null   int64  
 11  has_contract     7032 non-null   int64  
 12  is_male          7032 non-null   int64  
 13  has_partner      7032 non-null   int64  
 14  has_dep          7032 non-null   int64  
 15  tenure_months    7032 non-null   int64  
 16  has_phone        7032 non-null   int64  
 17  mult

In [58]:
df_1.nunique()

senior_citizen        2
tenure               72
monthly_charges    1584
total_charges      6530
auto_pay              2
dsl                   2
fiber                 2
has_internet          2
m2m                   2
one_year              2
two_year              2
has_contract          2
is_male               2
has_partner           2
has_dep               2
tenure_months        72
has_phone             2
multi_phone           2
has_security          2
has_backup            2
has_protection        2
has_support           2
stream_tv             2
stream_movies         2
has_paperless         2
has_churn             2
dtype: int64